# Scraping weather from LV
This report aims to scrape weather information from lv.fo for the various weather stations

## Setup

### Libraries

In [1]:
import pandas as pd
from numpy import nan
import openpyxl
import lxml 
import xml.sax
import requests
from datetime import date, timedelta
import glob
import os
import time


In [2]:
#conda install openpyxl
#conda install lxml 
#conda install lxml 

### Constants

In [3]:
start_date = date(2023,2,7)
end_date = date(2023,2,9)
# taken from https://www.lv.fo/fr/
weather_stations = {'F-40' : 'Dalavegur til Viðareiðis',
                    'F-24': 'Klaksvík, Gerðabøur',
                    'F-39': 'Gøtueiði',
                    'F-26': 'Syðradalur',
                    'F-10': 'Kambsdalur',
                    'F-22': 'Runavík',
                    'F-37': 'Norðskálatunnilin, Eystanfyri',
                    'F-42': 'Gjáarskarð',
                    'F-43': 'Heltnin, Oyndarfjarðarvegurin',
                    'F-12': 'Høgareyn',
                    'F-35': 'Tjørnuvíkarvegurin',
                    'F-45': 'Streymnes',
                    'F-36': 'Norðuri í Sundum, Kollafjørður',
                    'F-38': 'Kaldbaksbotnur',
                    'F-21': 'Sund',
                    'F-48': 'við Velbastaðháls',
                    'F-33': 'Norðradalsskarð',
                    'F-41': 'Á Sandavágshálsi',
                    'F-23': 'Vatnsoyrar',
                    'F-29': 'Skopunar havn',
                    'F-25': 'Sandoy, á Brekkuni Stóru',
                    'F-44': 'Hvalba',
                    'F-28': 'Krambatangi',
                    'F-27': 'Porkerishálsur',
                    'F-49': 'Ørðaskarð, Fámjinsvegurin'
                   }


OUTPUT_PATH = 'temp_Download'

### Helpers

#### Looping through dates

In [4]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)+1):
        yield start_date + timedelta(n)

#### XML Parser

In [5]:
def xml_to_dfs(path):
    """Read Excel XML file at path and return list of DataFrames"""

    class ExcelXMLHandler(xml.sax.handler.ContentHandler):
        def __init__(self):
            self.tables = []
            self.chars = []

        def characters(self, content):
            self.chars.append(content)

        def startElement(self, name, attrs):
            if name == "Table":
                self.rows = []
            elif name == "Row":
                self.cells = []
            elif name == "Data":
                self.chars = []

        def endElement(self, name):
            if name == "Table":
                self.tables.append(self.rows)
            elif name == "Row":
                self.rows.append(self.cells)
            elif name == "Data":
                self.cells.append("".join(self.chars))

    exh = ExcelXMLHandler()
    xml.sax.parse(path, exh)
    for table in exh.tables:

        #return [pd.DataFrame(table[1:][0:][0:58], columns=table[0][0:58]) for table in exh.tables]

        # Yuck this is bad practice!!! Pythonify this shit!!!!!
        for table in exh.tables:
            dat = table[1:][0:]
            col = table[0][0:58]
            datClean = []
            for d in dat:
                d = d[0:58]
                datClean.append(d)

        return [pd.DataFrame(datClean, columns=col)]

#df = xml_to_dfs(output_file_name)
#df

In [6]:
#output_file_name = temp_Download/2023_02_07_F-35.xml
#print(output_file_name)
#df = xml_to_dfs(output_file_name)

## Download files

In [7]:
for single_date in daterange(start_date, end_date):
    print("-----------------------")
    print(f'Parsing date {single_date.strftime("%Y-%m-%d")}')
    year = single_date.strftime("%Y")
    month = single_date.strftime("%m")
    day = single_date.strftime("%d")
    
    
    for w in weather_stations:
        station_id = w
        station_name = weather_stations[w]
        print(f'-  Checking {station_id} ({station_name})')

        download_path = f'https://lv.fo/fr/excel.php?station={station_id}&year={year}&month={month}&day={day}'
        output_file_name = OUTPUT_PATH + f'/{year}_{month}_{day}_{station_id}.xml'
        
        try: 
            # Download file
            r = requests.get(download_path, allow_redirects=True, timeout=5)

            # Save file
            open(output_file_name, 'wb').write(r.content)
            print(output_file_name)

            # Close connection
            r.close()
            #time.sleep(3)


            # Parse broken XML
            df = xml_to_dfs(output_file_name)

            # Select relevant columns
            data_df = df[0]
            # Make sure to create if missing
            data_df['time'] = data_df.get('time', nan)
            data_df['temp2'] = data_df.get('temp2', nan)
            data_df['hum'] = data_df.get('hum', nan)
            data_df['dew'] = data_df.get('dew', nan)
            data_df['rain'] = data_df.get('rain', nan)
            data_df['mean1'] = data_df.get('mean1', nan)
            data_df['dir'] = data_df.get('dir', nan)
            data_df['rainsum'] = data_df.get('rainsum', nan)
            data_df['rainint'] = data_df.get('rainint', nan)
            data_df['press1'] = data_df.get('press1', nan)
            data_df['rainclass'] = data_df.get('rainclass', nan)
            data_df['gust2'] = data_df.get('gust2', nan)
            data_df['road1_surface'] = data_df.get('road1_surface', nan)
            data_df['road1_freez'] = data_df.get('road1_freez', nan)
            data_df['road1_status'] = data_df.get('road1_status', nan)
            data_df['road1_salt'] = data_df.get('road1_salt', nan)
            data_df['road2_surface'] = data_df.get('road2_surface', nan)
            data_df['road2_freez'] = data_df.get('road2_freez', nan)
            data_df['road2_salt'] = data_df.get('road2_salt', nan)

            # Reduce subset
            data_df = data_df[['time', 'temp2', 'hum', 'dew', 'rain', 'mean1', 'dir',
                             'rainsum', 'rainint', 'press1', 'rainclass', 'gust2',
                             'road1_surface', 'road1_freez', 'road1_status', 'road1_salt',
                             'road2_surface', 'road2_freez','road2_salt'
                            ]]

            # Append identifiers
            c = {'date': single_date, 'station_id': station_id, 'station_name': station_name}
            results = data_df.assign(**c)
            # Save to file
            results.to_csv(f'cleansed_csvs/{year}_{month}_{day}_{station_id}.csv', encoding = "cp1252", index=False)
        except Exception:
            pass
    

-----------------------
Parsing date 2023-02-07
-  Checking F-40 (Dalavegur til Viðareiðis)
temp_Download/2023_02_07_F-40.xml
-  Checking F-24 (Klaksvík, Gerðabøur)
temp_Download/2023_02_07_F-24.xml
-  Checking F-39 (Gøtueiði)
temp_Download/2023_02_07_F-39.xml
-  Checking F-26 (Syðradalur)
temp_Download/2023_02_07_F-26.xml
-  Checking F-10 (Kambsdalur)
temp_Download/2023_02_07_F-10.xml
-  Checking F-22 (Runavík)
temp_Download/2023_02_07_F-22.xml
-  Checking F-37 (Norðskálatunnilin, Eystanfyri)
temp_Download/2023_02_07_F-37.xml
-  Checking F-42 (Gjáarskarð)
temp_Download/2023_02_07_F-42.xml
-  Checking F-43 (Heltnin, Oyndarfjarðarvegurin)
temp_Download/2023_02_07_F-43.xml
-  Checking F-12 (Høgareyn)
temp_Download/2023_02_07_F-12.xml
-  Checking F-35 (Tjørnuvíkarvegurin)
temp_Download/2023_02_07_F-35.xml
-  Checking F-45 (Streymnes)
temp_Download/2023_02_07_F-45.xml
-  Checking F-36 (Norðuri í Sundum, Kollafjørður)
temp_Download/2023_02_07_F-36.xml
-  Checking F-38 (Kaldbaksbotnur)
temp_

## Aggregate to single CSV

In [10]:
# Read in csvs
path = r'cleansed_csvs'
all_files = glob.glob(os.path.join(path , "*.csv"))
li = []

# Go through them
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, encoding = "cp1252")
    li.append(df)
    
# combine
frame = pd.concat(li, axis=0, ignore_index=True)
# cleanse
#frame = frame.drop(frame.columns[-1],axis=1)
# save
frame.to_csv(f'aggregated_{start_date.strftime("%Y-%m-%d")}-{end_date.strftime("%Y-%m-%d")}.csv', encoding = "cp1252", index=False)

In [13]:
frame

,time,temp2,hum,dew,rain,mean1,dir,rainsum,rainint,press1,...,road1_surface,road1_freez,road1_status,road1_salt,road2_surface,road2_freez,road2_salt,date,station_id,station_name
0,22:05,8.5,78.0,4.9,1.0,18.3,234.0,14.3,0.46,1558.5,...,5.6,0.0,13.0,0.0,5.3,-0.0,0.0,2023-02-07,F-10,Kambsdalur
1,22:00,7.6,86.0,5.4,1.0,16.3,228.0,14.3,0.42,1543.6,...,5.6,0.0,13.0,0.0,5.3,-0.0,0.0,2023-02-07,F-10,Kambsdalur
2,21:55,8.0,81.0,4.9,1.0,15.7,223.0,14.2,0.73,1551.3,...,5.5,0.0,13.0,0.0,5.2,-0.0,0.0,2023-02-07,F-10,Kambsdalur
3,21:50,8.4,78.0,4.9,1.0,16.5,222.0,14.2,0.70,1549.6,...,5.5,0.0,13.0,0.0,5.2,-0.0,0.0,2023-02-07,F-10,Kambsdalur
4,21:45,7.5,86.0,5.4,1.0,16.6,228.0,14.1,0.37,1548.1,...,5.4,0.0,13.0,0.0,5.1,-0.0,0.0,2023-02-07,F-10,Kambsdalur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16619,00:14,0.2,89.0,-1.5,0.0,19.2,225.0,2.7,0.00,961.4,...,-1.3,-6.7,24.0,3.3,-1.1,-5.2,2.6,2023-02-09,F-49,"Ørðaskarð, Fámjinsvegurin"
16620,00:10,0.0,87.0,-1.9,0.0,18.6,227.0,2.7,0.00,960.8,...,-1.3,-7.1,24.0,3.4,-1.0,-5.5,2.7,2023-02-09,F-49,"Ørðaskarð, Fámjinsvegurin"
16621,00:07,0.0,86.0,-2.0,0.0,17.9,228.0,2.7,0.00,961.1,...,-1.3,-7.5,24.0,3.6,-1.0,-5.8,2.9,2023-02-09,F-49,"Ørðaskarð, Fámjinsvegurin"
16622,00:04,-0.1,86.0,-2.2,0.0,17.7,230.0,2.7,0.00,961.5,...,-1.2,-7.9,24.0,3.8,-0.9,-6.0,3.0,2023-02-09,F-49,"Ørðaskarð, Fámjinsvegurin"
